# Validate configuration for reproducing "Revisiting TCP Congestion Control Throughput Models & Fairness Properties at Scale"

## Set up your FABRIC environment


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
fablib.show_config()

In [ ]:
!chmod 600 {fablib.get_bastion_key_filename()}
!chmod 600 {fablib.get_default_slice_private_key_file()}

## Get slice details

Put your slice name and the number of endpoints in the following cell:

In [ ]:
n_endpoints = 10
slice_name="bottleneck-" + str(n_endpoints) + '-test'

Then, load your slice details into the environment.slice = fablib.new_slice(name=slice_name)

In [ ]:
slice = fablib.get_slice(name=slice_name)

In [ ]:
sender_nodes = [slice.get_node(name='sender-' + str(i))  for i in range(n_endpoints)]
receiver_nodes = [slice.get_node(name='receiver-' + str(i))  for i in range(n_endpoints)]

In [ ]:
router_node = slice.get_node(name='router')
router_ingress_iface = router_node.get_interface(network_name = "link-sender")
router_egress_iface  = router_node.get_interface(network_name = "link-receiver")
router_egress_name = router_egress_iface.get_device_name()

## Check network capacity

We should get close to 25 Gbps (probably more like 23 Gbps because of protocol overhead) through the router.

First, make sure there is no rate limiting at the router - don't worry if you get an error message, this just means there was already no rate limiting.

In [ ]:
router_node.execute("sudo tc qdisc del dev " + router_egress_name + " root")

Change the queue at the sender nodes to pfifo_fast to replace existing fq_codel queue

In [ ]:
for n in sender_nodes:
    sender_inf=n.get_interface(network_name= "link-sender")
    sender_inf_name = sender_inf.get_device_name()
    n.execute("sudo tc qdisc replace dev " + sender_inf_name + " root handle 1: pfifo_fast")
    n.execute("tc -s -d qdisc show dev "+ sender_inf_name)


Now start the `iperf3` flows:

In [ ]:
for n in receiver_nodes:
    n.execute("sudo killall iperf3; iperf3 -s -1 -f g -D --logfile validate.dat")

In [ ]:
import time
for i,n in enumerate(sender_nodes):
    n.execute_thread("sudo killall iperf3; iperf3 -t 60 -c 10.10.2.1"+str(i))
time.sleep(185)

In [ ]:
tputs = []
for n in receiver_nodes:
    (nout, nerr) = n.execute("tail --lines=2 validate.dat | grep receiver | awk '{print $7}'")
    tputs.append(float(nout.strip()))
print("Sum throughput is: %f Gbps" % sum(tputs))

A single pair of endpoints should be able to get about 10 Gbps, possibly higher.

In [ ]:
receiver_nodes[0].execute("iperf3 -s -1 -f g -D")
sender_nodes[0].execute("iperf3 -t 60 -i 60 -c 10.10.2.10")

## Check network delay

We should get sub-ms latency across the router.

In [ ]:
for i,n in enumerate(sender_nodes):
    n.execute("ping -c 5 10.10.2.1"+str(i)+" | grep rtt")